In [ ]:
python -m pip install --upgrade pip

In [ ]:
!pip install ultralytics

In [ ]:
pip show ultralytics

In [ ]:
!pip install ultralytics

In [ ]:
import sys
print(sys.executable)

In [ ]:
pip uninstall ultralytics
pip install ultralytics

In [ ]:
pip install --upgrade ultralytics

In [1]:
import cv2
from ultralytics import YOLO
import time
import datetime

In [2]:
model = YOLO('best_violence_model.pt')

In [ ]:
# Define the class names
class_names = {
    0: 'weapon',
    1: 'abuse',
    2: 'violence',
    3: 'attack',
    4: 'harassment',
    5: 'chain snatching',
    6: 'rape',
    7: 'women-men-knife-gun'
}

# Initialize the webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Give the camera some time to warm up
time.sleep(2)

# Set the desired FPS for recording
input_fps = 30  # Frames per second (capturing speed)

# Major Changes Made below..................................................
output_fps = input_fps // 6  # Output FPS for slower video (6x slower) Takles the fast-forwared video issue ( hyper-laspe).

# Get frame size
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_size = (frame_width, frame_height)

# Recording settings
# Major Changes Made below..................................................
recording_duration = 30  # seconds ( balances out the output_fps effect ..... bruteforce hehehehe ... by trial and error .

# Initialize flags and tracking variables
recording = False
frames_written = 0
incident_active = False  # Set this when violence/weapon is detected
start_time = None

# Video writer
out = None

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame.")
        break

    # Convert frame to RGB for YOLO model input
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform object detection using YOLO
    results = model.predict(rgb_frame, device='cpu')

    # Check for violence-related detection
    violence_detected = False
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0]
            class_id = int(box.cls[0])  # Get the class ID

            # Mark as violence if class ID matches violence-related actions
            if class_id in [0, 1, 2, 3, 4, 6, 7]:
                violence_detected = True

            # Draw bounding box and label on the frame
            class_name = class_names.get(class_id, "Unknown")
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'{class_name} {confidence*100:.2f}%'
            cv2.putText(frame, label, (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Start recording if violence is detected (even for one frame)
    if violence_detected and not incident_active:
        incident_active = True
        recording = True
        frames_written = 0
        start_time = time.time()  # Record the time when recording starts

        # Generate unique filename using timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'output_clip_{timestamp}.webm'

        # Initialize VideoWriter object with the slower output FPS
        fourcc = cv2.VideoWriter_fourcc(*'vp90')  # WebM format
        out = cv2.VideoWriter(filename, fourcc, output_fps, frame_size)

        print(f"Recording started: {filename}")

    # If recording, write frames to the video
    if recording:
        out.write(frame)  # Write the current frame to the video file
        frames_written += 1
        print(f"Recording in progress... ")
        # Stop recording after real-time 15 seconds, regardless of detection status
        if time.time() - start_time >= recording_duration:
            recording = False
            incident_active = False  # Reset incident after the full recording
            out.release()
            print(f"Recording stopped. Video saved as {filename}")

    # Display the live webcam feed
    cv2.imshow('Webcam', frame)

    # Press 'q' to quit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
if out is not None and recording:
    out.release()  # Ensure the VideoWriter is released
cv2.destroyAllWindows()


0: 480x640 (no detections), 297.2ms
Speed: 10.1ms preprocess, 297.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 250.9ms
Speed: 0.0ms preprocess, 250.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 269.6ms
Speed: 0.0ms preprocess, 269.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 256.5ms
Speed: 0.0ms preprocess, 256.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 258.5ms
Speed: 0.0ms preprocess, 258.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 177.5ms
Speed: 2.3ms preprocess, 177.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 175.3ms
Speed: 0.0ms preprocess, 175.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 166.3ms
Speed: 1.0ms prep